In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
!pip install tensorflow

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import display, Javascript
from keras.utils import to_categorical

In [ ]:
# 런타임 오류 방지 함수
def keep_alive():
    display(Javascript('''
        function ClickConnect(){
            console.log("클릭 연결 버튼");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 60000)
    '''))

In [ ]:
# 데이터 로드 및 전처리
def load_and_preprocess_data():
    # '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/json to df.csv'에서 데이터를 로드
    df = pd.read_csv('/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/json to df.css')

    # 문자열로 저장된 딕셔너리를 실제 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

    # Training 데이터만 선택
    df = df[df['split'] == 'Training']
    return preprocess_data(df)

# 데이터 전처리
# annotations를 처리하여 리스트 형태의 값을 길이로 변환
# equipment에서 딕셔너리가 아닌 값은 빈 딕셔너리로 정리
def preprocess_data(df):
    def process_annotations(anno):
        if isinstance(anno, dict):
            return {k: len(v) if isinstance(v, list) else v for k, v in anno.items()}
        return {}

    df['annotations'] = df['annotations'].apply(process_annotations)
    df['equipment'] = df['equipment'].apply(lambda x: x if isinstance(x, dict) else {})
    return merge_eye_cheek_data(df)

# 눈가와 볼 데이터 합치기
# facepart 3,4를 34로, 5,6을 56으로 통일
def merge_eye_cheek_data(df):
    df.loc[df['images'].apply(lambda x: x['facepart'] in [3, 4]), 'images'] = df['images'].apply(lambda x: {**x, 'facepart': 34} if x['facepart'] in [3, 4] else x)
    df.loc[df['images'].apply(lambda x: x['facepart'] in [5, 6]), 'images'] = df['images'].apply(lambda x: {**x, 'facepart': 56} if x['facepart'] in [5, 6] else x)
    return df

In [ ]:
# 데이터 증강 함수
# 이미지에 상하 전환, 대비, 회전 적용
def augment_data(image):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    return image

# lr_poly 함수 정의
def lr_poly(initial_lr, iter, max_iter, power):
    return initial_lr * ((1 - float(iter) / max_iter) ** power)

# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = tf.keras.applications.resnet50.preprocess_input(img)
    return img

# 데이터 제너레이터 생성 함수
def create_data_generator(datagen, X, y, directory):
    df = pd.DataFrame({'filename': X})
    for col in y.columns:
        df[col] = y[col]

    return datagen.flow_from_dataframe(
        dataframe=df,
        directory=directory,
        x_col='filename',
        y_col=y.columns.tolist(),
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw'
    )

In [ ]:
# ResNet50 모델 생성
# 분류와 회귀를 위한 출력층 추가
def create_model(output_dims):
    # ImageNet 가중치로 초기화된 ResNet50 모델 생성
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    outputs = []
    for name, dim in output_dims.items():
        if dim == 1:  # 회귀 출력
            outputs.append(Dense(1, name=name)(x))
        else:  # 분류 출력
            outputs.append(Dense(dim, activation='softmax', name=name)(x))
    return Model(inputs=base_model.input, outputs=outputs)

In [ ]:
# 모델 학습
def train_model(model, train_data, val_data, output_dims, facepart, model_type, epochs=100, batch_size=32):
    # 하이퍼파라미터 설정
    initial_lr = 1e-3
    power = 0.9
    optimizer = Adam(learning_rate=initial_lr)

    # 손실 함수와 평가 지표 설정
    losses = {}
    metrics = {}
    for name, dim in output_dims.items():
        if dim == 1:  # 회귀
            losses[name] = 'mean_squared_error'
            metrics[name] = 'mae'
        else:  # 분류
            losses[name] = 'sparse_categorical_crossentropy'
            metrics[name] = 'accuracy'

    model.compile(optimizer=optimizer, loss=losses, metrics=metrics)

    # 체크포인트 설정 (10 에폭마다 저장)
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/facepart_{facepart}_{model_type}_checkpoint_{{epoch:02d}}.h5',
        save_best_only=True,
        save_weights_only=False,
        monitor='loss',
        mode='min',
        save_freq=10)

    # 모델 학습
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[
            tf.keras.callbacks.LearningRateScheduler(lambda epoch: lr_poly(initial_lr, epoch, epochs, power)),
            checkpoint_callback,
            tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)
        ])

    # 최종 모델 저장
    model.save(f'/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/facepart_{facepart}_{model_type}_final_model.h5')
    return history

In [ ]:
# 성능 시각화 함수
# 학습 과정의 손실과 성능을 그래프 시각화
def plot_performance(history, output_dims, facepart, model_type):
    n_metrics = len(output_dims)
    fig, axes = plt.subplots(n_metrics, 2, figsize=(15, 5*n_metrics))

    for i, (name, dim) in enumerate(output_dims.items()):
        # Loss plot
        axes[i, 0].plot(history.history[f'{name}_loss'], label='Train Loss')
        axes[i, 0].plot(history.history[f'val_{name}_loss'], label='Validation Loss')
        axes[i, 0].set_title(f'{name} Loss')
        axes[i, 0].set_xlabel('Epoch')
        axes[i, 0].set_ylabel('Loss')
        axes[i, 0].legend()

        # Metric plot
        metric = 'accuracy' if dim > 1 else 'mae'
        axes[i, 1].plot(history.history[f'{name}_{metric}'], label=f'Train {metric.upper()}')
        axes[i, 1].plot(history.history[f'val_{name}_{metric}'], label=f'Validation {metric.upper()}')
        axes[i, 1].set_title(f'{name} {metric.upper()}')
        axes[i, 1].set_xlabel('Epoch')
        axes[i, 1].set_ylabel(metric.upper())
        axes[i, 1].legend()

    plt.tight_layout()
    plt.savefig(f'/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/facepart_{facepart}_{model_type}_performance.png')
    plt.close()

In [ ]:
# 각 facepart에 대해 데이터를 준비하고 모델을 훈련
# 분류와 회귀 모델을 별도로 생성하고 훈련
# facepart별 모델 훈련
def train_facepart_models(facepart):
    print(f"Processing facepart {facepart}")

    # facepart에 따른 데이터 필터링
    if facepart in [34, 56]:
        facepart_df = df[df['images'].apply(lambda x: x['facepart'] in ([3, 4] if facepart == 34 else [5, 6]))]
    else:
        facepart_df = df[df['images'].apply(lambda x: x['facepart'] == facepart)]

    # bbox 유효성 검사 및 필터링
    def valid_bbox(bbox):
        if bbox is None:
            return False
        if isinstance(bbox, list) and len(bbox) == 4:
            if bbox == ['None', 'None', 'None', 'None']:
                return False
            return all(isinstance(b, int) and b > 0 for b in bbox)
        return False

    facepart_df = facepart_df[facepart_df['images'].apply(lambda x: valid_bbox(x.get('bbox')))]

    # 이미지 경로 설정
    image_directory = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training/01.원천데이터' if facepart == 0 else f'/gdrive/MyDrive/Final project/1_Red/4_데이터탐색_전처리/facepart별 피부 이미지/Training_cropped/{facepart}'

    X = facepart_df['info'].apply(lambda x: os.path.join(image_directory, x['filename'] if facepart == 0 else f"{x['filename'].split('.')[0]}_{facepart}.jpg"))

    # annotations 및 equipment 데이터 처리
    anno_columns = set().union(*facepart_df['annotations'])
    equip_columns = set().union(*facepart_df['equipment'])

    y_class = pd.DataFrame()
    y_reg = pd.DataFrame()
    output_dims_class = {}
    output_dims_reg = {}

    # 분류 데이터 준비
    for col in anno_columns:
        y_class[col] = facepart_df['annotations'].apply(lambda x: x.get(col, None))
        output_dims_class[col] = len(set(y_class[col].dropna())) if len(set(y_class[col].dropna())) > 1 else 1

    # 회귀 데이터 준비
    for col in equip_columns:
        y_reg[col] = facepart_df['equipment'].apply(lambda x: x.get(col, None))
        output_dims_reg[col] = 1  # 회귀

    # facepart 0에 대한 특별 처리
    if facepart == 0:
        y_class['skin_type'] = facepart_df['info'].apply(lambda x: x['skin_type'])
        y_class['sensitive'] = facepart_df['info'].apply(lambda x: x['sensitive'])
        output_dims_class['skin_type'] = len(set(y_class['skin_type'].dropna()))
        output_dims_class['sensitive'] = len(set(y_class['sensitive'].dropna()))
        y_class['acne_count'] = facepart_df['annotations'].apply(lambda x: len(x.get('acne', [])))
        output_dims_class['acne_count'] = 1  # 회귀

    # 분류 모델 학습
    if not y_class.empty:
        print(f"Starting classification training for facepart {facepart}")
        X_train, X_val, y_train_class, y_val_class = train_test_split(X, y_class, test_size=0.1, random_state=42)

        train_datagen = ImageDataGenerator(preprocessing_function=augment_data)
        val_datagen = ImageDataGenerator(preprocessing_function=load_and_preprocess_image)

        train_generator_class = create_data_generator(train_datagen, X_train, y_train_class, X_train.iloc[0].rsplit('/', 1)[0])
        val_generator_class = create_data_generator(val_datagen, X_val, y_val_class, X_val.iloc[0].rsplit('/', 1)[0])

        model_class = create_model(output_dims_class)
        history_class = train_model(model_class, train_generator_class, val_generator_class, output_dims_class, facepart, 'classification')
        plot_performance(history_class, output_dims_class, facepart, 'classification')

    # 회귀 모델 학습
    if not y_reg.empty and not all(y_reg.isnull().all()):
        print(f"Starting regression training for facepart {facepart}")
        X_train, X_val, y_train_reg, y_val_reg = train_test_split(X, y_reg, test_size=0.1, random_state=42)

        train_generator_reg = create_data_generator(train_datagen, X_train, y_train_reg, X_train.iloc[0].rsplit('/', 1)[0])
        val_generator_reg = create_data_generator(val_datagen, X_val, y_val_reg, X_val.iloc[0].rsplit('/', 1)[0])

        model_reg = create_model(output_dims_reg)
        history_reg = train_model(model_reg, train_generator_reg, val_generator_reg, output_dims_reg, facepart, 'regression')
        plot_performance(history_reg, output_dims_reg, facepart, 'regression')

In [ ]:
# 메인 실행
if __name__ == "__main__":
    keep_alive()
    df = load_and_preprocess_data()
    user_input = input("처리할 facepart 범위를 선택하세요 (1: 0-2, 2: 34,56, 3: 7-8): ")
    if user_input == '1':
        facepart_range = [0, 1, 2]
    elif user_input == '2':
        facepart_range = [34, 56]
    elif user_input == '3':
        facepart_range = [7, 8]
    else:
        print("잘못된 입력입니다.")
        exit()

    for facepart in facepart_range:
        train_facepart_models(facepart)

- 데이터 로드 및 전처리:
    - load_and_preprocess_data() 함수:
        - '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/json to df.csv'에서 데이터를 로드합니다.
        - 문자열로 저장된 딕셔너리를 실제 딕셔너리로 변환합니다.
        - Training 데이터만 선택합니다.
    - preprocess_data() 함수:
        - annotations를 처리하여 리스트 형태의 값을 길이로 변환합니다.
        - equipment 데이터를 정리합니다.
    - merge_eye_cheek_data() 함수:
        - facepart 3,4를 34로, 5,6을 56으로 합칩니다.


- 데이터 증강 및 전처리:
    - 이미지에 대해 상하 반전, 대비 조정, 회전을 적용하는 augment_data 함수를 정의합니다.
    - 이미지를 로드하고 전처리하는 load_and_preprocess_image 함수를 정의합니다.


- 데이터 제너레이터 생성:
    - create_data_generator 함수를 사용하여 Keras의 ImageDataGenerator를 생성합니다.


- facepart별 모델 훈련:
    - train_facepart_models 함수에서 각 facepart에 대한 모델을 훈련합니다.
    - annotations에 대한 분류 모델과 equipment에 대한 회귀 모델을 생성합니다.
    - facepart 0의 경우, info 컬럼의 'skin_type'과 'sensitive'에 대한 추가 분류 모델을 생성합니다.


- 모델 훈련:
    - train_model 함수에서 실제 모델 훈련이 이루어집니다.
    - 학습률 스케줄링, 체크포인트 저장, 조기 종료 등의 콜백을 사용합니다.
    - 모델은 '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/' 경로에 저장됩니다.


- 성능 시각화:
    - plot_performance 함수에서 훈련 및 검증 손실, 정확도 또는 MAE를 시각화합니다.
    - 시각화 결과는 '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/' 경로에 저장됩니다.


- 메인 실행:
    - 사용자 입력에 따라 처리할 facepart 범위를 선택합니다.
    - 선택된 facepart에 대해 train_facepart_models 함수를 실행합니다.